In [1]:
import openai
import json
from sklearn.metrics import classification_report
import pandas as pd
from tqdm import tqdm
from credentials import OPENAPI_KEY
openai.api_key = OPENAPI_KEY

In [ ]:
train_ds_path = "DS_PATH/train.jsonl"
openai.File.create(file=open(train_ds_path, "rb"), purpose="fine-tune")

In [ ]:
openai.FineTuningJob.create(training_file="FILE_ID", model="gpt-3.5-turbo")

In [ ]:
openai.FineTuningJob.retrieve("FINETUNE_ID")

In [4]:
test_ds_path = "DS_PATH/test.jsonl"
with open(test_ds_path, "r") as f:
    test_dataset = [json.loads(line) for line in f]

In [5]:
test_dataset[0]["messages"]

[{'role': 'system',
  'content': 'VRL-Detector is a chatbot that can detect if the given Turkish text contains Hate Speech.'},
 {'role': 'user',
  'content': 'Does this Turkish text contain hate speech? : "suriyeli çetelerin evlilik vaadiyle kandırıp binlerce tl dolandırılan cahillere zerre kadar üzülmüyorum ...türkiye\'de kadın kalmadı de mi? beter olun... 15 ocak çarşamba" '},
 {'role': 'assistant', 'content': 'yes'}]

In [22]:
preds = []
refs = []
for example in tqdm(test_dataset[1095:]):
    completion = openai.ChatCompletion.create(
        model="MODEL_ID",
        messages=example["messages"][:2]
        )
    model_output = completion.choices[0].message
    ground_truth = example["messages"][-1]["content"]
    preds.append(model_output["content"])
    refs.append(ground_truth)

100%|██████████| 4759/4759 [28:25<00:00,  2.79it/s]


In [29]:
output_df = pd.DataFrame({"preds": preds, "refs": refs})
output_df.to_csv("OUTPUT_PATH/output.csv", index=False)

In [30]:
output_labels = ["no", "yes"]
for index in range(len(preds)):
    if preds[index] not in output_labels:
        preds[index] = "no" 

In [31]:
print(classification_report(refs, preds, digits=4))

              precision    recall  f1-score   support

          no     0.8848    0.8477    0.8658      4366
         yes     0.6020    0.6761    0.6369      1488

    accuracy                         0.8041      5854
   macro avg     0.7434    0.7619    0.7514      5854
weighted avg     0.8129    0.8041    0.8076      5854



In [32]:
set(preds)

{'no', 'yes'}

In [136]:
temp_df  = pd.read_csv("OUTPUT_PATH/output.csv")

temp_df.head()

,preds,refs
0,positive,neutral
1,neutral,neutral
2,neutral,neutral
3,neutral,neutral
4,neutral,negative
